In [1]:
import requests
from bs4 import BeautifulSoup
from io import StringIO
import csv

URL = "https://www.amfiindia.com/downloads"

response = requests.get(URL)
soup = BeautifulSoup(response.content, 'html.parser')
links = soup.find_all('a')

flag = 0
csv_url =""
for i in links:
    if 'Scheme Data Download' in i.text:
        csv_url = i['href']
        flag = 1
        break

if flag == 0:
    print("Error while scraping")
else:
    print("Scraped successfully!")
    response = requests.get(csv_url)
    content = response.content.decode('utf-8')
    csv_reader = csv.reader(StringIO(content))

    with open('Schemes.csv', 'w', newline='') as file:
        csv_writer = csv.writer(file)
        for row in csv_reader:
            csv_writer.writerow(row)
    

Scraped successfully!


In [2]:
import pandas as pd

In [115]:
new_df = pd.read_csv("./Schemes.csv")
old_df = pd.read_csv("./Scheme-Master.csv")

In [116]:
data = new_df[~new_df['Code'].isin(old_df['AMFICode'])]\
# data = pd.read_csv('./SchemeData1908231158SS.csv')

In [109]:
def scheme_category(scheme):
    list_ = [i.replace("Scheme","").replace("&", "and").strip() for i in scheme.split(' - ')]
    if len(list_) < 2:
        return ["", list_[0]]
    return list_


def ISIN(x):
    isin_payout = ""
    isin_reinv = ""
    if '-' in x:
        x = x.split('-')
    else:
        x = x.split(' ')
    x = [i.strip() for i in x if len(i)>=12]
    if len(x)==1:
        x=x[0]
        if len(x)==24:
            isin_payout = x[0:12]
            isin_reinv = x[12:24]
        elif len(x)==12:
            isin_payout = x
    elif len(x)==2:
        if len(x[0]) == 12:
            isin_payout = x[0]
        if len(x[1]) == 12:
            isin_reinv = x[1]
    return [isin_payout, isin_reinv]

    return x


def amount(x):
    min_amount = ''
    x+=' '
    for i in range(len(x)):
        if x[i].isdigit():
            for k in x[i:]:
                if k.isdigit():
                    min_amount += k
                else:
                    return int(min_amount)
    return None


In [110]:
bubbleData = pd.DataFrame({'AMC': data['AMC'], "AMFICode": data['Code'], 'SchemeName': data['Scheme Name'], 'SchemeType': data['Scheme Type'], 'AMFISchemeCategory': data['Scheme Category']})
bubbleData[["SchemeCategory", "SubCategory"]] = bubbleData['AMFISchemeCategory'].apply(scheme_category).apply(pd.Series)
bubbleData['SchemeNAVName'] = data['Scheme NAV Name']
bubbleData['Fundtype'] = bubbleData['SubCategory'].apply(lambda x: 'FoF' if 'FoF' in x else 'DE')
bubbleData['PlanType'] = bubbleData['SchemeNAVName'].apply(lambda x: 'Direct' if 'direct' in x.lower() else 'Regular')
bubbleData[["ISINDivPayout_ISIN Growth", "ISINDivReinvestment"]] = data['ISIN Div Payout/ ISIN GrowthISIN Div Reinvestment'].apply(str).apply(ISIN).apply(pd.Series)
bubbleData['LaunchDate'] = data['Launch Date']
bubbleData['ClosureDate'] = data[' Closure Date']
bubbleData['AMFIMininmumAmount'] = data['Scheme Minimum Amount']
bubbleData['SchemeMinimumAmount'] = data['Scheme Minimum Amount'].apply(str).apply(amount)
bubbleData['isPrimary'] = bubbleData['SchemeNAVName'].apply(lambda x: 'Y' if 'growth' in x.lower() or 'regular' in x.lower() else 'N')

In [111]:
# old_df.append(bubbleData)
# new_df2.to_csv('Scheme-Master.csv', index=False)
new_df2 = pd.concat([old_df, bubbleData], ignore_index=True)

In [67]:
out = bubbleData.to_dict(orient='records')

In [ ]:
import 

In [70]:
type(jsonify(out))

NameError: name 'jsonify' is not defined